In [1]:
import MySQLdb

In [2]:
db = MySQLdb.connect(host="localhost",  # host      # 127.0.0.1
                     user="root",       # username  # root
                     passwd="1234",     # password  # 1234
                     db="naver_movies")         # DB NAME

In [3]:
cur = db.cursor()

In [4]:
db

<_mysql.connection open to 'localhost' at 46d2168>

In [5]:
cur

In [7]:
# IF(조건, 참값, 거짓값)
query = """
        CREATE TABLE IF NOT EXISTS naver_movie
            (code    INTEGER      NOT NULL      PRIMARY KEY,
             url     VARCHAR(255) NOT NULL,
             created_at    DATETIME DEFAULT CURRENT_TIMESTAMP,
             title   VARCHAR(100) NOT NULL,
             story   TEXT,
             opened  DATETIME)
             ENGINE INNODB;
        """
cur.execute(query)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: Warning: (1050, "Table 'naver_movie' already exists")
  if sys.path[0] == '':


0

In [8]:
query = """
        CREATE TABLE IF NOT EXISTS movie_comment(
            star    INTEGER    NOT NULL,
            comment TEXT       NOT NULL,
            movie_code INTEGER,
            author  VARCHAR(100),
            commented_at DATETIME,
            created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
            PRIMARY KEY (author, commented_at),
            FOREIGN KEY(movie_code)
            REFERENCES naver_movie(code) on delete cascade)
            ENGINE INNODB;
            ;
"""
cur.execute(query)

0

In [9]:
import requests
from urllib import parse
from bs4 import BeautifulSoup
import datetime
import math

In [10]:
def get_movie_list(start_page, end_page):
    today = datetime.datetime.now()
    
    movie_list = []
    for page in range(start_page, end_page+1):
        movie_list_url = 'https://movie.naver.com/movie/sdb/rank/rmovie.nhn'
        params = {
            'sel':'pnt',
            'date':today.strftime('%Y%m%d'),
            'page':page
        }
        resp = requests.get(movie_list_url)
        resp.encoding = 'utf-8'
    
        soup = BeautifulSoup(resp.content)
        # list_ranking = soup.find(class_='list_ranking').find_all('tr')
        list_ranking = soup.select('.list_ranking>tbody>tr')
    
        for movie_tr in list_ranking:
            all_td = movie_tr.find_all('td')
            try:
                img = all_td[0].img['alt']
                link = all_td[1].find('a')['href']
                title = all_td[1].find('a').text
                
                code = parse.parse_qs(parse.urlsplit(link).query)['code'][0]
            except TypeError:
                pass

            else:
                movie_list.append({
                    'img':img,
                    'link':link,
                    'title':title,
                    'code':code
                })
                
    return movie_list

In [16]:
def get_movie_detail(code):
    base_url = 'https://movie.naver.com/movie/bi/mi/basic.nhn'
    params = {
        'code':code
    }
    resp = requests.get(base_url, params)
    resp.encoding ='utf-8'
    soup = BeautifulSoup(resp.content)
    
    opend = soup.find('dt', text='개요()').find_next_sibling('dd').find_all('span')[-1].text.strip()
    opend = ''.join(opend.split('\n'))[:10]

    story = soup.find('div', class_='story_area')
    print(story)
    return {
        'opend':opend,
        'story':story
    }
    
    
def get_movie_comment(code, last_comment='15788558'):
    base_url = '/movie/bi/mi/pointWriteFormList.nhn'
    page = 1
    comment_list = []
    
    while True:
        params={
            'code':171539,
            'type':'after',
            'order':'newest',
            'page':page
        }
        resp = requests.get(base_url, params)
        resp.encoding = 'utf-8'
        soup = BeautifulSoup(resp.content)
        
        comments = soup.find('div', class_='score_result').find_all('li')
        for comment in comments:
            star = comment.find('div', class_='star_score').text
            reple = comment.find('div', class_='score_reple').find('p').text
            
            dd = comment.find('div', class_='score_reple').find('dl').find_all('dd')
            user = dd[0]
            created_at = dd[1]
            
            comment_id = comment.find('div', class_='btn_area').find_all('span')[-1]['class'][0].split('_')[1]
            
            if int(comment_id) <= int(last_comment):
                return comment_list
            
            comment_list.append({
                'star':star,
                'reple':reple,
                'user': user,
                'created_at': created_at,
                'comment_id': comment_id
            })
            
        
        page += 1

In [13]:
# movie_list

# 한 화면에 50페이지
PAGE_SIZE = 50
MOVIE_LENGTH = 100
pages = math.ceil(MOVIE_LENGTH/PAGE_SIZE)
pages

2

In [14]:
movie_list = get_movie_list(1, pages)

In [25]:
len(movie_list)

98

In [19]:
movie_list[]

<div class="story_area">
<div class="title_area">
<h4 class="h_story"><strong class="blind">줄거리</strong></h4>
</div>
<h5 class="h_tx_story">언제 어디서든 바른 생활! 완벽한 천재 뮤지션 ‘돈 셜리’<br/>
원칙보다 반칙! 다혈질 운전사 ‘토니’<br/>
취향도, 성격도 완벽히 다른 두 남자의 특별한 우정이 시작된다!</h5>
<br/> 그들을 위한 여행안내서 ‘그린북’에 의존해 특별한 남부 투어를 시작하는데…</p>
<button class="story_more" id="toggleMakingnoteButton" onclick="storyAndNote.toggleMakingnote();" type="button"><em class="blind">제작노트 보기</em></button><!-- N=a:mai.story -->
</div>


In [24]:
print(a['story'].text)



줄거리

언제 어디서든 바른 생활! 완벽한 천재 뮤지션 ‘돈 셜리’
원칙보다 반칙! 다혈질 운전사 ‘토니’
취향도, 성격도 완벽히 다른 두 남자의 특별한 우정이 시작된다!
 그들을 위한 여행안내서 ‘그린북’에 의존해 특별한 남부 투어를 시작하는데…
제작노트 보기

